In [7]:
from pprint import pprint
import json
import sys
sys.path.insert(0, '../genepattern-python/')
from gp import gp_functions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
gp_functions.login('https://genepattern.broadinstitute.org/gp', 'Simpli', 'Simpli')

gp_functions.get_tasks()

['ABSOLUTE',
 'ABSOLUTE.review',
 'ABSOLUTE.summarize',
 'Acgh2Tab',
 'AddFCSEventIndex',
 'AddFCSParameter',
 'AddNoiseToFCS',
 'AffySTExpressionFileCreator',
 'ApplyGatingML',
 'ARACNE',
 'AreaChange',
 'Arff2Gct',
 'ATARiS',
 'AuDIT',
 'BedToGtf',
 'Beroukhim.Getz.2007.PNAS.Glioma.GISTIC',
 'BlastTrainTest',
 'BlastXValidation',
 'Bowtie.aligner',
 'Bowtie.indexer',
 'BWA.aln',
 'BWA.bwasw',
 'BWA.indexer',
 'CaArray2ImportViewer',
 'CART',
 'CARTXValidation',
 'CBS',
 'ChIPSeq.CreateHeatmap',
 'ChIPSeq.CreatePlots',
 'ChIPSeq.ExtractSignal',
 'ClassNeighbors',
 'ClsFileCreator',
 'CNToXCN',
 'CoClustering',
 'CollapseDataset',
 'ComBat',
 'CombineOdf',
 'ComparativeMarkerSelection',
 'ComparativeMarkerSelectionViewer',
 'CompareSpectra',
 'CompensateFCS',
 'ConcatenateFilelist',
 'ConsensusClustering',
 'ConstellationMap',
 'ConvertLineEndings',
 'ConvertToMAGEML',
 'ConvertToMAGETAB',
 'COPA',
 'CopyNumberDivideByNormals',
 'CopyNumberInferencePipeline',
 'CopyNumberNoise',
 'copy

In [14]:
t = gp_functions.GPTask(gp_functions.SERVER, 'PreprocessDataset')
t.param_load()

required_args = []
optional_args = []
other_information = {}
task_entry = {
    "function_path": 'gp.gp_functions.execute',
    "label": t.get_name(),
    "description": t.get_description(),
    "required_args": required_args,
    "optional_args": optional_args,
    "other_information": other_information,
}
tasks = []
task_json = {
    "library_path": "Jumis/tools/genepattern-python",
    "tasks": tasks
}

ps = t.get_parameters()
for p in ps:
    a = {}
    description = p.get_description()
    default_value = p.get_default_value()
    
    a['arg_name'] = p.get_name()
    a['description'] = description
    if default_value:
        a['value'] = default_value
    
    type_ = p.get_type()
    if p.is_choice_param():
        choices = [d['value'] for d in p.get_choices()]
        a['description']: '{} (default={}) (choices={}) (type=)'.format(description, choices, default_value, type_,)
    else:
        a['description']: '{} (default={}) (type=)'.format(description, default_value, type_,)
    
    if default_value:
        optional_args.append(a)
    else:
        required_args.append(a)

other_information['lsid'] = t.get_lsid()
other_information['version'] = t.get_version()

tasks.append(task_entry)

pprint(task_json)
with open('/Users/kateme/Jumis/tools/genepattern-python/gp_functions.json', 'w') as f:
    json.dump(task_json, f, indent=4)

{'library_path': 'Jumis/tools/genepattern-python',
 'tasks': [{'description': 'Performs several preprocessing steps on a res, '
                           'gct, or odf input file',
            'function_path': 'gp.gp_functions.execute',
            'label': 'PreprocessDataset',
            'optional_args': [{'arg_name': 'threshold.and.filter',
                               'description': 'Apply floor, ceiling and '
                                              'variation filter',
                               'value': '1'},
                              {'arg_name': 'floor',
                               'description': 'Value for floor threshold',
                               'value': '20'},
                              {'arg_name': 'ceiling',
                               'description': 'Value for ceiling threshold',
                               'value': '20000'},
                              {'arg_name': 'min.fold.change',
                               'description': 'Mini

In [ ]:
js.set_parameter('input.filename',
                 'https://software.broadinstitute.org/cancer/software/genepattern/data/all_aml/all_aml_test.gct')

In [ ]:
j = gp_functions.submit_job(js)

In [ ]:
j.get_status_message()

In [ ]:
j.is_finished()

In [ ]:
for f in j.get_output_files():
    print(f.get_url())
    data = f.read()

In [ ]:
data